In [ ]:
import tweepy
import json
import numpy as np
import pandas as pd

# APIを使用するための鍵
consumer_key="YOUR KEY"
consumer_secret="YOUR KEY"
access_token="YOUR KEY"
access_token_secret="YOUR KEY"

# TweepyによるTwitter APIを使用するための認証のセットアップ
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# パラメータwait_on_rate_limitを有効化して、 APIの問い合わせ回数の上限に達した場合は必要時間だけ待機
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
# 検索ワードとして exploit http を使用し、リツイートは除外
search_term = "exploit http -filter:retweets"

# 一番古いツイートidを保存するための変数
oldest_tweet = None

# 回収したツイートの一覧を格納するためのリスト
TempDict = []

# 回収したツイート数のカウンター
counter = 0

# １０回ループし、合計1000件のツイートを対象にする
for x in range(10):

    # 最新のツイートから100件を抽出し、テキストをすべて収集する
    public_tweets = api.search(search_term, 
                               count=100, 
                               result_type="recent",
                               tweet_mode="extended", 
                               max_id=oldest_tweet)

    # 条件に一致するツイートの収集
    for tweet in public_tweets["statuses"]:
        #　引用リツイートも除外
        if not 'quoted_status' in tweet:
            TempDict.append(tweet)
                            
            # カウンターに１を追加
            counter += 1

        # 検索結果の一番古いツイートidを代入し、次の検索結果はこの一番古いid
        # より古いものだけを対象にする
        oldest_tweet = tweet["id"]

print("Tweet {}件を収集しました".format(counter))

Tweet 9件を収集しました


In [ ]:
data = pd.DataFrame(data=[tweet['full_text'] for tweet in TempDict], columns=['TweetText'])

In [ ]:
import re
URLPATTERN = r'(https?://\S+)'

data['URL'] = data["TweetText"].str.extract(URLPATTERN, expand=False).str.strip()

In [ ]:
data.head(20)

,TweetText,URL
0,OL : L'exploit face à Man City élu match de l'...,https://t.co/Ih2fyz2b9j
1,Republican efforts to undermine Biden victory ...,https://t.co/h8r9kOXBuY
2,The scattershot efforts to overturn President-...,https://t.co/BubYeURzb2
3,"@MSNBC Only goes to show, how #DonaldTrump wil...",https://t.co/PwzukzW05L
4,"@MSNBC Only goes to show, how #DonaldTrump wil...",https://t.co/PwzukzW05L
5,exploit frictionless #supply-chains with #doma...,https://t.co/qTvwSQLAvj
6,#DeFi insurer @CoverProtocol was exploited for...,https://t.co/hlS98EtpPw
7,Thanks for TAB message format. I did some expl...,https://t.co/SZUogOi0Lj
8,"I ruin credit scores, marriages &amp; manipula...",https://t.co/NYQPND6xUo


In [ ]:
!pip install pigeonXT-jupyter

In [ ]:
# pandasの列幅を300に設定してツイート本文を確認できるよう設定を変更
pd.set_option("display.max_colwidth", 300)

In [ ]:
from pigeonXT import annotate

# 一行ずつツイート本文の内容とURLを表示
annotations = annotate(data.index, options=['Exploit', 'NOT Exploit'], display_fn=lambda idx : display(data.loc[idx,['TweetText']],data.loc[idx,['URL']]))

HTML(value='0 of 9 Examples annotated, Current Position: 0 ')

Output()

Annotation done.


In [ ]:
# ラベル付けしたデータセットはJSONなので、行列に変換
dataset = pd.DataFrame(annotations.items(), columns=['index', 'label'])

# ツイート本文とURLの列をスクレイピング結果の行列からコピー
dataset['TweetText'] = data['TweetText']
dataset['URL'] = data['URL']

In [ ]:
dataset

,index,label,TweetText,URL
0,0,NOT Exploit,OL : L'exploit face à Man City élu match de l'année ! - https://t.co/Ih2fyz2b9j https://t.co/JPbXdxYUVl #OL #TeamOL #Gones,https://t.co/Ih2fyz2b9j
1,1,NOT Exploit,"Republican efforts to undermine Biden victory expose growing anti-democratic streak https://t.co/h8r9kOXBuY CONGRESS HAVE TO PASS LAWS THAT WILL REQUIRE PROOF OF CHARGES AND A HARSH PENALTY FOR THOSE THAT WISH TO EXPLOIT THIS SITUATION AGAIN, IF THEY LOSE FOR PROTECTION!!",https://t.co/h8r9kOXBuY
2,2,NOT Exploit,The scattershot efforts to overturn President-elect Joe Biden's election victory are coalescing into a movement led by top Republicans determined to exploit a manufactured crisis for broader political gains.\n \n https://t.co/BubYeURzb2,https://t.co/BubYeURzb2
3,3,NOT Exploit,"@MSNBC Only goes to show, how #DonaldTrump will ever continue to exploit his personal assessment of the #RepublicanParty @GOP; nuff said!\n\nhttps://t.co/PwzukzW05L",https://t.co/PwzukzW05L
4,4,NOT Exploit,"@MSNBC Only goes to show, how #DonaldTrump will ever continue to exploit his personal assessment of the #RepublicanParty @GOP; nuff said!\n\nhttps://t.co/PwzukzW05L",https://t.co/PwzukzW05L
5,5,NOT Exploit,exploit frictionless #supply-chains with #domains \n💰 Buy #domain #covid #lockdown https://t.co/qTvwSQLAvj https://t.co/aScsQgLhMq https://t.co/CyVH9mIsUy,https://t.co/qTvwSQLAvj
6,6,NOT Exploit,#DeFi insurer @CoverProtocol was exploited for $4.4 million this week — raising concerns about investor protection. Learn more about the state of decentralized insurance in this week's edition of DeFi Digest👇\nhttps://t.co/hlS98EtpPw https://t.co/4aOfLWedf8,https://t.co/hlS98EtpPw
7,7,NOT Exploit,Thanks for TAB message format. I did some exploit? https://t.co/SZUogOi0Lj https://t.co/JkK0MqRjL8 and not support,https://t.co/SZUogOi0Lj
8,8,NOT Exploit,"I ruin credit scores, marriages &amp; manipulate weak men for fun, I like to exploit their weaknesses for Me &amp; then leave them high &amp; dry.\n\nKeep sending until you're broke. \nhttps://t.co/NYQPND6xUo https://t.co/MylCf74B7I",https://t.co/NYQPND6xUo


In [ ]:
dataset.to_csv('dataset.csv', index=False)